# PREPARING THE DATA

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("icici/ICICIBANK.NS.csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("icici/icici_bank_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
# slicing the data
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
sent_model = tf.keras.models.load_model("final_bert")
# removing duplicates from the data
final_posts.drop_duplicates(subset=['Messages'])
# calculating the sentiments score
sentiments = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        _,msg,time = j
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)
# getting indices where sentiments score is 0
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
# removing all the zero values indices
sentiments = np.delete(sentiments,zero_index)
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
# removing the added helper column
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

In [2]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
final_x_zeros_scaled = mms.fit_transform(final_x_zeros)
final_x_zeros_scaled

array([[0.46964598, 0.7110796 , 0.830842  , ..., 0.47434157, 0.5546069 ,
        0.7407407 ],
       [0.49432564, 0.72036934, 0.819458  , ..., 0.37842518, 0.53776145,
        0.7037037 ],
       [0.5260892 , 0.73180676, 0.8039284 , ..., 0.33568484, 0.5818709 ,
        0.5925926 ],
       ...,
       [0.9687238 , 0.98316383, 0.7430401 , ..., 0.23337865, 0.32989419,
        0.25925925],
       [0.965055  , 0.9778986 , 0.73816586, ..., 0.21261415, 0.42023087,
        0.25925925],
       [0.96308565, 0.94291973, 0.68118954, ..., 0.34671447, 0.65432334,
        0.25925925]], dtype=float32)

In [3]:
final_y_zeros

,Close
Date,
2021-11-16,764.049988
2021-11-17,761.299988
2021-11-18,762.900024
2021-11-22,752.000000
2021-11-23,751.849976
2021-11-24,760.200012
2021-11-25,751.049988
2021-11-26,722.200012
2021-11-29,718.400024


In [4]:
k = final_y_zeros.max()
final_y_zeros= final_y_zeros/k
final_y_zeros

,Close
Date,
2021-11-16,0.926458
2021-11-17,0.923124
2021-11-18,0.925064
2021-11-22,0.911847
2021-11-23,0.911665
2021-11-24,0.921790
2021-11-25,0.910695
2021-11-26,0.875712
2021-11-29,0.871105


# GETTING THE NBEATS FOR COMPARISON

In [5]:
model_nbeats = tf.keras.models.load_model("nbeats_icici")
model_nbeats.evaluate(final_x_zeros[43:],final_y_zeros[43:]*k)

1/1 [==============================] - 1s 654ms/step - loss: 10.6543 - mae: 10.6543 - mse: 175.6807


[10.654330253601074, 10.654330253601074, 175.6807098388672]

# BUILDING AND TRAINING THE MODEL

In [12]:
tf.random.set_seed(42)
inputs = tf.keras.layers.Input(shape=(71))
x = tf.keras.layers.Lambda(lambda x : tf.expand_dims(x,axis=1))(inputs)
x = tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.LSTM(128)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs = inputs, outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 71)]              0         
                                                                 
 lambda_1 (Lambda)           (None, 1, 71)             0         
                                                                 
 lstm_5 (LSTM)               (None, 1, 128)            102400    
                                                                 
 lstm_6 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_7 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_8 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_9 (LSTM)               (None, 128)               1315

In [13]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
history = model.fit(final_x_zeros_scaled[:43],final_y_zeros[:43],
                    epochs = 1000,
                    validation_data = (final_x_zeros_scaled[43:], final_y_zeros[43:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 1/1000
2/2 [==============================] - 6s 1s/step - loss: 0.9149 - mae: 0.9149 - mse: 0.8386 - val_loss: 0.9581 - val_mae: 0.9581 - val_mse: 0.9181 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 57ms/step - loss: 0.9040 - mae: 0.9040 - mse: 0.8188 - val_loss: 0.9455 - val_mae: 0.9455 - val_mse: 0.8941 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 55ms/step - loss: 0.8914 - mae: 0.8914 - mse: 0.7962 - val_loss: 0.9297 - val_mae: 0.9297 - val_mse: 0.8644 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 57ms/step - loss: 0.8760 - mae: 0.8760 - mse: 0.7689 - val_loss: 0.9081 - val_mae: 0.9081 - val_mse: 0.8247 - lr: 0.0010
Epoch 5/1000
2/2 [==============================] - 0s 58ms/step - loss: 0.8556 - mae: 0.8556 - mse: 0.7334 - val_loss: 0.8761 - val_mae: 0.8761 - val_mse: 0.7678 - lr: 0.0010
Epoch 6/1000
2/2 [==============================] - 0s 52ms/step - loss: 0.8270 - mae: 0.8270 - mse: 0.6851 - val_loss: 0.

Epoch 47/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0258 - mae: 0.0258 - mse: 0.0010 - val_loss: 0.0163 - val_mae: 0.0163 - val_mse: 3.9053e-04 - lr: 0.0010
Epoch 48/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0245 - mae: 0.0245 - mse: 8.9507e-04 - val_loss: 0.0188 - val_mae: 0.0188 - val_mse: 4.9027e-04 - lr: 0.0010
Epoch 49/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0200 - mae: 0.0200 - mse: 6.4352e-04 - val_loss: 0.0207 - val_mae: 0.0207 - val_mse: 5.8149e-04 - lr: 0.0010
Epoch 50/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0208 - mae: 0.0208 - mse: 7.6719e-04 - val_loss: 0.0245 - val_mae: 0.0245 - val_mse: 7.6816e-04 - lr: 0.0010
Epoch 51/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0276 - mae: 0.0276 - mse: 0.0012 - val_loss: 0.0123 - val_mae: 0.0123 - val_mse: 2.2742e-04 - lr: 0.0010
Epoch 52/1000
2/2 [==============================] - 0s 36ms/step - loss: 0.0238 - 

Epoch 92/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0261 - mae: 0.0261 - mse: 0.0011 - val_loss: 0.0126 - val_mae: 0.0126 - val_mse: 1.9292e-04 - lr: 0.0010
Epoch 93/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0299 - mae: 0.0299 - mse: 0.0012 - val_loss: 0.0128 - val_mae: 0.0128 - val_mse: 1.9356e-04 - lr: 0.0010
Epoch 94/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0224 - mae: 0.0224 - mse: 9.3971e-04 - val_loss: 0.0449 - val_mae: 0.0449 - val_mse: 0.0022 - lr: 0.0010
Epoch 95/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0248 - mae: 0.0248 - mse: 8.1443e-04 - val_loss: 0.0443 - val_mae: 0.0443 - val_mse: 0.0021 - lr: 0.0010
Epoch 96/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0244 - mae: 0.0244 - mse: 8.7407e-04 - val_loss: 0.0127 - val_mae: 0.0127 - val_mse: 1.9203e-04 - lr: 0.0010
Epoch 97/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0238 - mae: 0.0

2/2 [==============================] - 0s 34ms/step - loss: 0.0225 - mae: 0.0225 - mse: 7.5912e-04 - val_loss: 0.0291 - val_mae: 0.0291 - val_mse: 0.0011 - lr: 0.0010
Epoch 137/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0175 - mae: 0.0175 - mse: 4.6862e-04 - val_loss: 0.0217 - val_mae: 0.0217 - val_mse: 6.5779e-04 - lr: 0.0010
Epoch 138/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0146 - mae: 0.0146 - mse: 3.3937e-04 - val_loss: 0.0267 - val_mae: 0.0267 - val_mse: 9.6287e-04 - lr: 0.0010
Epoch 139/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0141 - mae: 0.0141 - mse: 4.4209e-04 - val_loss: 0.0219 - val_mae: 0.0219 - val_mse: 6.6947e-04 - lr: 0.0010
Epoch 140/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0149 - mae: 0.0149 - mse: 4.2476e-04 - val_loss: 0.0157 - val_mae: 0.0157 - val_mse: 2.8402e-04 - lr: 0.0010
Epoch 141/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0235 - mae: 

Epoch 179/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0191 - mae: 0.0191 - mse: 5.4811e-04 - val_loss: 0.0262 - val_mae: 0.0262 - val_mse: 9.3367e-04 - lr: 1.0000e-04
Epoch 180/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0139 - mae: 0.0139 - mse: 3.0346e-04 - val_loss: 0.0261 - val_mae: 0.0261 - val_mse: 9.3177e-04 - lr: 1.0000e-04
Epoch 181/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0191 - mae: 0.0191 - mse: 5.7344e-04 - val_loss: 0.0258 - val_mae: 0.0258 - val_mse: 9.1126e-04 - lr: 1.0000e-04
Epoch 182/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0149 - mae: 0.0149 - mse: 3.4888e-04 - val_loss: 0.0255 - val_mae: 0.0255 - val_mse: 8.9554e-04 - lr: 1.0000e-04
Epoch 183/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0152 - mae: 0.0152 - mse: 3.6724e-04 - val_loss: 0.0267 - val_mae: 0.0267 - val_mse: 9.6574e-04 - lr: 1.0000e-04
Epoch 184/1000
2/2 [==============================

2/2 [==============================] - 0s 33ms/step - loss: 0.0161 - mae: 0.0161 - mse: 4.0554e-04 - val_loss: 0.0199 - val_mae: 0.0199 - val_mse: 5.1690e-04 - lr: 1.0000e-04
Epoch 223/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0161 - mae: 0.0161 - mse: 4.0828e-04 - val_loss: 0.0224 - val_mae: 0.0224 - val_mse: 6.9978e-04 - lr: 1.0000e-04
Epoch 224/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0146 - mae: 0.0146 - mse: 3.2066e-04 - val_loss: 0.0253 - val_mae: 0.0253 - val_mse: 8.8388e-04 - lr: 1.0000e-04
Epoch 225/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0164 - mae: 0.0164 - mse: 4.1840e-04 - val_loss: 0.0273 - val_mae: 0.0273 - val_mse: 0.0010 - lr: 1.0000e-04
Epoch 226/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0161 - mae: 0.0161 - mse: 3.7800e-04 - val_loss: 0.0269 - val_mae: 0.0269 - val_mse: 9.8505e-04 - lr: 1.0000e-04
Epoch 227/1000
2/2 [==============================] - 0s 34ms/step - 

# FINE TUNING THE MODEL

In [8]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["mae","mse"])
history = model.fit(final_x_zeros_scaled[:43],final_y_zeros[:43],
                    epochs = 2000,
                    initial_epoch = history.epoch[-1],
                    validation_data = (final_x_zeros_scaled[43:], final_y_zeros[43:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 252/2000
2/2 [==============================] - 6s 1s/step - loss: 0.0252 - mae: 0.0252 - mse: 0.0010 - val_loss: 0.0183 - val_mae: 0.0183 - val_mse: 4.5804e-04 - lr: 1.0000e-04
Epoch 253/2000
2/2 [==============================] - 0s 53ms/step - loss: 0.0241 - mae: 0.0241 - mse: 9.3979e-04 - val_loss: 0.0268 - val_mae: 0.0268 - val_mse: 8.7666e-04 - lr: 1.0000e-04
Epoch 254/2000
2/2 [==============================] - 0s 44ms/step - loss: 0.0312 - mae: 0.0312 - mse: 0.0015 - val_loss: 0.0299 - val_mae: 0.0299 - val_mse: 0.0010 - lr: 1.0000e-04
Epoch 255/2000
2/2 [==============================] - 0s 44ms/step - loss: 0.0257 - mae: 0.0257 - mse: 0.0012 - val_loss: 0.0231 - val_mae: 0.0231 - val_mse: 6.9156e-04 - lr: 1.0000e-04
Epoch 256/2000
2/2 [==============================] - 0s 51ms/step - loss: 0.0218 - mae: 0.0218 - mse: 7.0300e-04 - val_loss: 0.0176 - val_mae: 0.0176 - val_mse: 4.3053e-04 - lr: 1.0000e-04
Epoch 257/2000
2/2 [==============================] - 0s 50ms/step -

2/2 [==============================] - 0s 34ms/step - loss: 0.0198 - mae: 0.0198 - mse: 7.1953e-04 - val_loss: 0.0193 - val_mae: 0.0193 - val_mse: 5.1144e-04 - lr: 1.0000e-04
Epoch 296/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0236 - mae: 0.0236 - mse: 7.9373e-04 - val_loss: 0.0179 - val_mae: 0.0179 - val_mse: 4.5217e-04 - lr: 1.0000e-04
Epoch 297/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0244 - mae: 0.0244 - mse: 8.9679e-04 - val_loss: 0.0185 - val_mae: 0.0185 - val_mse: 4.7606e-04 - lr: 1.0000e-04
Epoch 298/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0198 - mae: 0.0198 - mse: 7.6213e-04 - val_loss: 0.0186 - val_mae: 0.0186 - val_mse: 4.8203e-04 - lr: 1.0000e-04
Epoch 299/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0238 - mae: 0.0238 - mse: 9.0146e-04 - val_loss: 0.0193 - val_mae: 0.0193 - val_mse: 5.1287e-04 - lr: 1.0000e-04
Epoch 300/2000
2/2 [==============================] - 0s 33ms/ste

2/2 [==============================] - 0s 38ms/step - loss: 0.0199 - mae: 0.0199 - mse: 8.4114e-04 - val_loss: 0.0256 - val_mae: 0.0256 - val_mse: 8.5850e-04 - lr: 1.0000e-04
Epoch 339/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0179 - mae: 0.0179 - mse: 4.5256e-04 - val_loss: 0.0253 - val_mae: 0.0253 - val_mse: 8.4237e-04 - lr: 1.0000e-04
Epoch 340/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0237 - mae: 0.0237 - mse: 7.9350e-04 - val_loss: 0.0243 - val_mae: 0.0243 - val_mse: 7.8242e-04 - lr: 1.0000e-04
Epoch 341/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0211 - mae: 0.0211 - mse: 7.2542e-04 - val_loss: 0.0209 - val_mae: 0.0209 - val_mse: 6.0317e-04 - lr: 1.0000e-04
Epoch 342/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.0198 - mae: 0.0198 - mse: 5.7914e-04 - val_loss: 0.0154 - val_mae: 0.0154 - val_mse: 3.3399e-04 - lr: 1.0000e-04
Epoch 343/2000
2/2 [==============================] - 0s 37ms/ste

2/2 [==============================] - 0s 33ms/step - loss: 0.0247 - mae: 0.0247 - mse: 0.0010 - val_loss: 0.0230 - val_mae: 0.0230 - val_mse: 7.0762e-04 - lr: 1.0000e-05
Epoch 381/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0192 - mae: 0.0192 - mse: 6.1060e-04 - val_loss: 0.0230 - val_mae: 0.0230 - val_mse: 7.1064e-04 - lr: 1.0000e-05
Epoch 382/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0207 - mae: 0.0207 - mse: 6.8268e-04 - val_loss: 0.0229 - val_mae: 0.0229 - val_mse: 7.0694e-04 - lr: 1.0000e-05
Epoch 383/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0213 - mae: 0.0213 - mse: 6.9750e-04 - val_loss: 0.0229 - val_mae: 0.0229 - val_mse: 7.0271e-04 - lr: 1.0000e-05
Epoch 384/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0237 - mae: 0.0237 - mse: 8.6093e-04 - val_loss: 0.0227 - val_mae: 0.0227 - val_mse: 6.9532e-04 - lr: 1.0000e-05
Epoch 385/2000
2/2 [==============================] - 0s 33ms/step - 

2/2 [==============================] - 0s 33ms/step - loss: 0.0189 - mae: 0.0189 - mse: 5.8168e-04 - val_loss: 0.0254 - val_mae: 0.0254 - val_mse: 8.5003e-04 - lr: 1.0000e-05
Epoch 424/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0218 - mae: 0.0218 - mse: 8.1614e-04 - val_loss: 0.0256 - val_mae: 0.0256 - val_mse: 8.6195e-04 - lr: 1.0000e-05
Epoch 425/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0227 - mae: 0.0227 - mse: 7.5977e-04 - val_loss: 0.0258 - val_mae: 0.0258 - val_mse: 8.7186e-04 - lr: 1.0000e-05
Epoch 426/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.0209 - mae: 0.0209 - mse: 7.4196e-04 - val_loss: 0.0262 - val_mae: 0.0262 - val_mse: 8.9413e-04 - lr: 1.0000e-05
Epoch 427/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.0198 - mae: 0.0198 - mse: 7.1800e-04 - val_loss: 0.0266 - val_mae: 0.0266 - val_mse: 9.1671e-04 - lr: 1.0000e-05
Epoch 428/2000
2/2 [==============================] - 0s 36ms/ste

In [14]:
preds = tf.squeeze(model.predict(final_x_zeros_scaled[43:])).numpy()
preds = preds*k.values[0]
preds

array([811.1143 , 808.73364, 802.76324, 799.2455 , 799.18695, 801.7036 ,
       803.1478 , 802.50586, 795.94183, 798.50183, 803.0028 ],
      dtype=float32)

In [15]:
mae = sum(abs(preds-(tf.squeeze(final_y_zeros[43:].to_numpy()).numpy())*k.values[0]))
mae

107.05517578125

In [16]:
(tf.squeeze(final_y_zeros[43:].to_numpy()).numpy())*k.values[0]

array([801.65, 794.65, 781.15, 788.8 , 810.3 , 813.75, 808.95, 805.05,
       792.5 , 802.8 , 790.8 ], dtype=float32)

In [11]:
model.save("LSTM_icici")

INFO:tensorflow:Assets written to: LSTM_icici\assets


INFO:tensorflow:Assets written to: LSTM_icici\assets
